# Export Hospital Level Dashboards

**Purpose:**

1. **Optionally authenticate** with `gcloud` (via user prompt).
2. **Query BigQuery** for facility names (`faci_name` from `drg-viz.00_datasets.hci`).
3. **(Optional) Save** these names to a local CSV for debugging.
4. **Launch Selenium** (using `a_source_chrome_webdriver.py`) to open Power BI in **headless** mode,  
   select each facility in the slicer, and **export** a PDF to `./hospital-reports`.


In [ ]:
# Uncomment this to install prerequisites
# !pip install google-cloud-bigquery google-api-core selenium requests

import os
import sys
import json

# Third-party imports
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError

# Local imports (the new approach)
project_root = os.path.expanduser("~/pids-drg-hospital-dashboards")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.functions import *

# Define the path for saving adaptive wait times
WAIT_TIMES_JSON = os.path.join("..", "data", "cache", "final_wait_times.json")

def main():
    """
    Main function that:
      1) Loads (or initializes) the WAIT_TIMES.
      2) In debug mode, runs a calibration routine that tunes one wait time
         at a time (processing one hospital) to determine the minimal wait time needed.
      3) In non-debug mode, it uses the saved wait times and processes hospitals as usual.
      4) The final wait times are saved to WAIT_TIMES_JSON.
    """
    # Load wait times if the JSON exists; otherwise, initialize to 10 seconds for all.
    if os.path.isfile(WAIT_TIMES_JSON):
        with open(WAIT_TIMES_JSON, "r") as f:
            wait_times = json.load(f)
        print(f"Loaded WAIT_TIMES from {WAIT_TIMES_JSON}: {wait_times}")
    else:
        wait_times = {
            "iframe_wait": 10,
            "dropdown_sleep": 10,
            "search_sleep": 10,
            "visual_update_sleep": 10,
            "webdriver_wait_first_result": 10
        }
        print("No existing wait times found; initializing WAIT_TIMES to 10 for all steps.")
    
    # Update the global WAIT_TIMES in functions.py
    from src.functions import WAIT_TIMES
    WAIT_TIMES.clear()
    WAIT_TIMES.update(wait_times)
    
    # If in debug mode, calibrate each wait time one by one.
    # The calibration process runs test runs on a single hospital.
    if TO_DEBUG:
        print("\nStarting calibration of wait times...")
        calibrate_wait_times()
    else:
        # In non-debug mode, run a full attempt.
        attempt_run()

    # Save the final wait times to JSON
    os.makedirs(os.path.dirname(WAIT_TIMES_JSON), exist_ok=True)
    with open(WAIT_TIMES_JSON, "w") as f:
        json.dump(WAIT_TIMES, f)
    print(f"\nSaved final WAIT_TIMES to {WAIT_TIMES_JSON}.")

if __name__ == "__main__":
    main()


Loaded WAIT_TIMES from ../data/cache/final_wait_times.json: {'iframe_wait': 1, 'dropdown_sleep': 9, 'search_sleep': 7, 'visual_update_sleep': 9, 'webdriver_wait_first_result': 10}
Loaded 1885 entries from ../data/inputs/hospitals.csv.
Skipped header row in CSV.
Invalid input. Using all hospitals.
Processing 1884 hospitals.
Using 20 workers to process 1884 hospitals.
[Worker 6] Starting. Handling 95 hospital(s).
[Worker 4] Starting. Handling 95 hospital(s).
[Worker 3] Starting. Handling 95 hospital(s).
Local Chrome version: 133.0.6943.142
Detected driver platform: mac-arm64
Local Chrome version: 133.0.6943.142
Detected driver platform: mac-arm64
[Worker 1] Starting. Handling 95 hospital(s).
Local Chrome version: 133.0.6943.142
Detected driver platform: mac-arm64
[Worker 5] Starting. Handling 95 hospital(s).
Local Chrome version: 133.0.6943.142
Detected driver platform: mac-arm64
[Worker 2] Starting. Handling 95 hospital(s).
[Worker 7] Starting. Handling 95 hospital(s).
Closest known-goo